In [ ]:
%pip install matplotlib torchinfo

In [ ]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

In [ ]:
import json

with open("../db-dump-1745150542.json") as f:
    data = json.load(f)

assert len(data["temperature"]) >= 360, "Temperature data is less than 360 entries"
assert len(data["humidity"]) >= 360, "Humidity data is less than 360 entries"

In [ ]:
import numpy as np

temp_values = np.array([item["value"] for item in data["temperature"]])
temp_times = np.array([item["ts"] for item in data["temperature"]])
humi_values = np.array([item["value"] for item in data["humidity"]])
humi_times = np.array([item["ts"] for item in data["humidity"]])

In [ ]:
import math

times_interpolated = np.linspace(
    temp_times[0], temp_times[-1], math.ceil((temp_times[-1] - temp_times[0]) / 5)
)

temp_interpolated = np.interp(times_interpolated, temp_times, temp_values)
humi_interpolated = np.interp(times_interpolated, humi_times, humi_values)

In [ ]:
import matplotlib.pyplot as plt
from numpy.typing import NDArray


def draw_diff_scatter(
    times: NDArray[np.float32],
    values: NDArray[np.float32],
    interp_times: NDArray[np.float32],
    interp_values: NDArray[np.float32],
):
    latest_time = times[-1]
    plt.figure(figsize=(10, 6))
    plt.scatter(times - latest_time, values, label="Original Data", marker="o")
    plt.scatter(interp_times - latest_time, interp_values, label="Interpolated Data", marker="x")
    plt.xlabel("Time (relative to latest) [sec]")
    plt.ylabel("Value")
    plt.title("Original vs Interpolated Data")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
draw_diff_scatter(humi_times, humi_values, times_interpolated, humi_interpolated)

In [ ]:
draw_diff_scatter(temp_times, temp_values, times_interpolated, temp_interpolated)

In [ ]:
from torch.utils.data import Dataset
import torch
from typing import override


class DHTDataset(Dataset[tuple[torch.Tensor, torch.Tensor]]):
    def __init__(self, data: list[float]):
        self.inputs = []
        self.target = []

        for i in range(36, len(data) - 120):
            self.inputs.append(data[i - 36 : i])
            self.target.append(data[i + 120])

    def __len__(self):
        return len(self.inputs)

    @override
    def __getitem__(self, index: int):
        return (
            torch.tensor(self.inputs[index]).float(),
            torch.tensor(self.target[index]).float(),
        )

In [ ]:
temp_train = DHTDataset(temp_interpolated[: len(temp_interpolated) - 180])
temp_test = DHTDataset(temp_interpolated[len(temp_interpolated) - 180 :])

humi_train = DHTDataset(humi_interpolated[: len(humi_interpolated) - 180])
humi_test = DHTDataset(humi_interpolated[len(humi_interpolated) - 180 :])

In [ ]:
from torch.utils.data import DataLoader

temp_train_loader = DataLoader(temp_train, batch_size=32, shuffle=True)
temp_test_loader = DataLoader(temp_test, batch_size=32, shuffle=False)
humi_train_loader = DataLoader(humi_train, batch_size=32, shuffle=True)
humi_test_loader = DataLoader(humi_test, batch_size=32, shuffle=False)

In [ ]:
import random

print(f"temperature train data length: {len(temp_train)}")
print(f"temperature test data length: {len(temp_test)}")
print(f"temperature data example: {temp_train[random.randint(0, len(temp_train))]}")
print(f"temperature batch shape: {next(iter(temp_train_loader))[0].shape}")

print(f"humidity train data length: {len(humi_train)}")
print(f"humidity test data length: {len(humi_test)}")
print(f"humidity data example: {humi_train[random.randint(0, len(humi_train))]}")
print(f"humidity batch shape: {next(iter(humi_train_loader))[0].shape}")

In [ ]:
from iot_lab_base.engine.ex9_temp import Ex9Model

temp_model = Ex9Model(36, 32, 3)
humi_model = Ex9Model(36, 32, 3)

In [ ]:
import torchinfo

torchinfo.summary(
    temp_model,
    input_data=next(iter(temp_test_loader))[0],
    col_names=("input_size", "output_size", "num_params"),
    row_settings=("depth", "var_names"),
)

In [ ]:
from typing import Any
import torch.nn as nn
import torch.optim as optim


def train_model(model: Ex9Model, num_epochs: int, train_loader: DataLoader[Any]):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        for train_data, train_target in train_loader:
            outputs = model(train_data)
            loss = criterion(outputs, train_target.unsqueeze(1))
            loss.backward()
            optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
    model.eval()
    return model

In [ ]:
NUM_EPOCHS = 1000

print("[Temperature model]")
train_model(temp_model, NUM_EPOCHS, temp_train_loader)
print("\n[Humidity model]")
train_model(humi_model, NUM_EPOCHS, humi_train_loader)

print("\nFinished training")

In [ ]:
def test_model(model: Ex9Model, test_loader: DataLoader[Any]):
    criterion = nn.MSELoss()
    model.eval()
    with torch.no_grad():
        for test_data, test_target in test_loader:
            outputs = model(test_data)
            loss = criterion(outputs, test_target.unsqueeze(1))
            print(f"Test Loss: {loss.item():.4f}")

In [ ]:
print("[Temperature]")
test_model(temp_model, temp_test_loader)

print("\n[Humidity]")
test_model(humi_model, humi_test_loader)

In [ ]:
torch.save(temp_model.state_dict(), "../ex9_temp.ckpt")
torch.save(humi_model.state_dict(), "../ex9_humi.ckpt")